In [1]:
#using Pkg
#Pkg.add("Gumbo")
#Pkg.add("Cascadia")
#Pkg.add("JSON")

In [2]:
using HTTP
using Gumbo
using Cascadia
using Dates
using JSON

In [3]:
# Yahoo!ニュースでのカテゴリー
topics_tags = ["domestic", "world", "business", "entertainment", "sports", "it", "science", "local"]

8-element Vector{String}:
 "domestic"
 "world"
 "business"
 "entertainment"
 "sports"
 "it"
 "science"
 "local"

In [4]:
base_url = "https://news.yahoo.co.jp/topics/"

"https://news.yahoo.co.jp/topics/"

In [5]:
# トピックス一覧から、タイトルとURL（概要）、投稿日時を取得
news_list_p = []
for category in topics_tags
    topics_url = base_url * category

    # 指定トピックのニュース一覧を取得
    response = HTTP.request("GET", topics_url)
    code = String(response.body)
    doc = Gumbo.parsehtml(code)
    #head = doc.root[1]
    body = doc.root[2]
    items = eachmatch(Selector(".newsFeed_item"), body) ;
    for item in items
        # url 
        link_items = eachmatch(Selector(".newsFeed_item_link"), item)
        if length(link_items) == 0   # 広告をスキップする
            continue
        end
        url = getattr(link_items[1], "href", nothing)
        # title
        title = nodeText(eachmatch(Selector(".newsFeed_item_title"), item)[1])
        # datetime
        dt = nodeText(eachmatch(Selector("time"), item)[1])
        result = Dict("url" => url, "title" => title, "datetime" => dt, "category" => category)
        push!(news_list_p, result)
    end
end

In [6]:
news_list_p

195-element Vector{Any}:
 Dict("datetime" => "6/30(木) 7:59", "title" => "NATO理事会 首相定期参加の意向", "url" => "https://news.yahoo.co.jp/pickup/6431068", "category" => "domestic")
 Dict("datetime" => "6/30(木) 7:51", "title" => "参院選 財源確保の議論は深まらず", "url" => "https://news.yahoo.co.jp/pickup/6431069", "category" => "domestic")
 Dict("datetime" => "6/30(木) 7:27", "title" => "きょうも熱中症警戒 広範囲で猛暑", "url" => "https://news.yahoo.co.jp/pickup/6431064", "category" => "domestic")
 Dict("datetime" => "6/30(木) 0:06", "title" => "節電 識者「政府は伝え方工夫を」", "url" => "https://news.yahoo.co.jp/pickup/6431058", "category" => "domestic")
 Dict("datetime" => "6/29(水) 22:07", "title" => "日米韓首脳会談 5年ぶりに実施", "url" => "https://news.yahoo.co.jp/pickup/6431052", "category" => "domestic")
 Dict("datetime" => "6/29(水) 21:06", "title" => "LGBT差別冊子 自民議連で配布か", "url" => "https://news.yahoo.co.jp/pickup/6431048", "category" => "domestic")
 Dict("datetime" => "6/29(水) 20:39", "title" => "批判に非難 財源を巡る論戦が過熱", "url" => "https://news.yahoo.co.jp

In [7]:
# URL（概要）のページから、URL（全文）を取得
news_list_d = []
# コラムなどは書き方が異なるので除外する
effective_url_base = r"https://news.yahoo.co.jp/articles/"
for ns in news_list_p
    response = HTTP.request("GET", ns["url"])
    ns_code = String(response.body)
    ns_doc = Gumbo.parsehtml(ns_code)
    body = ns_doc.root[2]
    detail_part = eachmatch(Selector("a[data-ual-gotocontent=\"true\"]"), body)
    if length(detail_part) > 0
        detail_url = getattr(detail_part[1], "href", nothing)
        if occursin(effective_url_base, detail_url)
            ns["url"] = detail_url
            push!(news_list_d, ns)
        end
    end
end

In [8]:
news_list_d

188-element Vector{Any}:
 Dict("datetime" => "6/30(木) 7:59", "title" => "NATO理事会 首相定期参加の意向", "url" => "https://news.yahoo.co.jp/articles/eed22b5a99572a2421f51b9d6c5b5c7c1cb08c47", "category" => "domestic")
 Dict("datetime" => "6/30(木) 7:51", "title" => "参院選 財源確保の議論は深まらず", "url" => "https://news.yahoo.co.jp/articles/03cc088b0b7d43b3a7977cd59b7885b15f0d49bc", "category" => "domestic")
 Dict("datetime" => "6/30(木) 7:27", "title" => "きょうも熱中症警戒 広範囲で猛暑", "url" => "https://news.yahoo.co.jp/articles/5f05514c32c452b09bf6ee80bfc0b23f197f779c", "category" => "domestic")
 Dict("datetime" => "6/30(木) 0:06", "title" => "節電 識者「政府は伝え方工夫を」", "url" => "https://news.yahoo.co.jp/articles/af1a36397e3da4afed61486c64888ef12693fed9", "category" => "domestic")
 Dict("datetime" => "6/29(水) 22:07", "title" => "日米韓首脳会談 5年ぶりに実施", "url" => "https://news.yahoo.co.jp/articles/4670f156d6c0949e8f3b00ee3114b65d8027af6e", "category" => "domestic")
 Dict("datetime" => "6/29(水) 20:39", "title" => "批判に非難 財源を巡る論戦が過熱", "url" 

In [9]:
# URL（本文）のページから、本文と、ニュースソースを取得
news_list = []
for ns in news_list_d
    response = HTTP.request("GET", ns["url"])
    code = String(response.body)
    doc = Gumbo.parsehtml(code)
    body = doc.root[2]
    source_parts = eachmatch(Selector("article > div.article_body > p"), body)
    source = length(source_parts) != 0 ? nodeText(source_parts[1]) : ""
    detail = nodeText(eachmatch(Selector("article > div.article_body > div> p"), body)[1])
    ns["detail"] = detail
    ns["source"] = source
    push!(news_list, ns)
end

In [10]:
news_list

188-element Vector{Any}:
 Dict("datetime" => "6/30(木) 7:59", "source" => "", "detail" => "　【マドリード時事】岸田文雄首相は29日午後（日本時間同日夜）、スペインの首都マドリードで開かれている北大西洋条約機構（NATO）首脳会議に、日本の首相として初めて出席した。\n\n　首相は、NATOの最高意思決定機関である北大西洋理事会（NAC）に定期参加したいとの意向を表明。覇権主義的な動きを強める中国を念頭に、東アジアの安全保障環境への危機感を訴え、認識の共有を促した。\n\n　同日の会合には、日本、韓国、オーストラリア、ニュージーランドが「アジア太平洋パートナー国」として招待された。首相は、この4カ国について「理事会会合への定期的な参加を進めていくべきだ」と強調した。\n\n　また、日本とNATOの具体的な協力を定めた2014年の「日・NATO国別パートナーシップ協力計画（IPCP）」を抜本的に改定すると表明。サイバー、新興技術、海洋安保などの分野で、協力を進展させる意向を伝えた。NATO本部への自衛官派遣や、軍事演習への相互オブザーバー参加などに取り組む方針も示した。　", "title" => "NATO理事会 首相定期参加の意向", "url" => "https://news.yahoo.co.jp/articles/eed22b5a99572a2421f51b9d6c5b5c7c1cb08c47", "category" => "domestic")
 Dict("datetime" => "6/30(木) 7:51", "source" => "", "detail" => "　参院選の各党の公約には、防衛費増額をはじめ少子化対策などでも歳出拡大につながる項目が並ぶ。\n\n　与党の自民、公明両党は財政健全化よりも物価高対策やコロナ禍からの経済回復を優先させる姿勢を強めている。野党では、国民民主党やれいわ新選組が国債発行による「積極財政」を前面に掲げているほか、各党とも物価高対策として消費税減税・廃止を主張。一方で、歳出拡大に見合う財源確保の議論は深まっていない。\n\n　◇「国債」頼み\n　自民は選挙公約に、防衛費増額のほか、脱炭素分野の成長に向け、新たな国債「GX経

In [11]:
# 保存
filename = "yahoo" * Dates.format(now(), "yyyymmdd") * ".txt"
open(filename, "w") do f
    println(f, json(news_list))
end

In [12]:
# 読み込み
data = JSON.parsefile("yahoo20220629.txt")

188-element Vector{Any}:
 Dict{String, Any}("datetime" => "6/30(木) 7:59", "source" => "", "detail" => "　【マドリード時事】岸田文雄首相は29日午後（日本時間同日夜）、スペインの首都マドリードで開かれている北大西洋条約機構（NATO）首脳会議に、日本の首相として初めて出席した。\n\n　首相は、NATOの最高意思決定機関である北大西洋理事会（NAC）に定期参加したいとの意向を表明。覇権主義的な動きを強める中国を念頭に、東アジアの安全保障環境への危機感を訴え、認識の共有を促した。\n\n　同日の会合には、日本、韓国、オーストラリア、ニュージーランドが「アジア太平洋パートナー国」として招待された。首相は、この4カ国について「理事会会合への定期的な参加を進めていくべきだ」と強調した。\n\n　また、日本とNATOの具体的な協力を定めた2014年の「日・NATO国別パートナーシップ協力計画（IPCP）」を抜本的に改定すると表明。サイバー、新興技術、海洋安保などの分野で、協力を進展させる意向を伝えた。NATO本部への自衛官派遣や、軍事演習への相互オブザーバー参加などに取り組む方針も示した。　", "title" => "NATO理事会 首相定期参加の意向", "url" => "https://news.yahoo.co.jp/articles/eed22b5a99572a2421f51b9d6c5b5c7c1cb08c47", "category" => "domestic")
 Dict{String, Any}("datetime" => "6/30(木) 7:51", "source" => "", "detail" => "　参院選の各党の公約には、防衛費増額をはじめ少子化対策などでも歳出拡大につながる項目が並ぶ。\n\n　与党の自民、公明両党は財政健全化よりも物価高対策やコロナ禍からの経済回復を優先させる姿勢を強めている。野党では、国民民主党やれいわ新選組が国債発行による「積極財政」を前面に掲げているほか、各党とも物価高対策として消費税減税・廃止を主張。一方で、歳出拡大に見合う財源確保の議論は深まっていない。\n\n　◇「国債」頼み\n　自民は選挙公約に、防衛費増